# Proof of concept for accessing data via TfL APIs and calculating distance between user and cycle hire stations in Python

### Import libraries

In [1]:
import requests
import json

### Access air quality APIs

In [2]:
# Get all bike points from TfL API
app_id = "your_app_id"
app_key = "your_app_key"
params = {'app_id': app_id, 'app_key': app_key}
r = requests.get('https://api.tfl.gov.uk/AirQuality', params=params)
air_quality = r.json()

In [3]:
# Today's forecast
air_quality['currentForecast'][0]

{u'$id': u'2',
 u'$type': u'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
 u'forecastBand': u'Moderate',
 u'forecastID': u'12421',
 u'forecastSummary': u'Moderate ozone air pollution forecast valid from Saturday 17 June to end of Monday 19 June GMT',
 u'forecastText': u'Saturday will be dry and sunny for most of the day, feeling warm with periods of cloud cover. A relatively clean air feed from the Atlantic should help dispersion of local emissions and pollution is likely to remain &#39;Low&#39;.&lt;br/&gt;&lt;br/&gt;A fine settle conditions forecast for Sunday and Monday with sunshine for  both days. Very light westerly winds will start to arrive from the east by Sunday evening, importing pollution from the north of the Continent. This air feed, combined with hours of unbroken sunshine will allow ozone production to build through both days, with &#39;Moderate&#39; ozone likely in the afternoons, particularly in suburban and outer-London locations.&lt;b

In [4]:
# Tomorrow's forecast
air_quality['currentForecast'][1]

{u'$id': u'3',
 u'$type': u'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
 u'forecastBand': u'Low',
 u'forecastID': u'12447',
 u'forecastSummary': u'Low air pollution forecast valid from Tuesday 20 June to end of Tuesday 20 June GMT',
 u'forecastText': u'Tuesday is forecast to be a hot sunny day, although with some cloud present and more wind activity, it&#39;s expected to been a little cooler than the last few days. A light easterly breeze is expected to increase moderate to fresh, from the afternoon onwards.&lt;br/&gt;&lt;br/&gt;During the first part of the day we are likely to import air which had previously been over the near continent. Later on, wind speeds are expected to increase and the path of incoming air change, so we will import relatively &#39;clean&#39; air from the North Sea. &lt;br/&gt;&lt;br/&gt;The change in the path of incoming air combined with some increased cloud and increased wind speed later, should ensure that although elevated 

### Define logic for calculating distance between coordinates

In [5]:
# Excerpt from 'haversine.py' (https://gist.github.com/rochacbruno/2883505)
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [6]:
# Test logic with TicketMaster office coordinates
origin = (51.5320521, -0.1092356)
destination = (51.529163, -0.10997)
distance(origin, destination)

0.3252451958196465

### Request cycle hire station data from TfL API

In [7]:
r = requests.get('https://api.tfl.gov.uk/BikePoint', params=params)
bike_points = r.json()

In [8]:
stations = []
for point in bike_points:
    row = {}
    for e in point['additionalProperties']:
        if e['key'] == 'NbDocks':
            row['docks'] = int(e['value'])
    row['name'] = point['commonName']
    row['id'] = point['id']
    row['lat'] = point['lat']
    row['lon'] = point['lon']
    stations.append(row)

In [9]:
# Export station data to JSON
data = {'objects': stations}
# with open('TfL_bike_station_data.json', 'wb') as outfile:
#    json.dump(data, outfile)

### Fetch user's three closest hire stations

In [10]:
distances = []
for row in data['objects']:
    entry = {}
    entry[distance(origin, (row['lat'], row['lon']))] = row['id']
    distances.append(entry)

In [11]:
distances = sorted(distances)

In [12]:
results = []
for e in distances[0:3]:
    for key, value in e.items():
        results.append(value)

### Print live hire station data from TfL API

In [13]:
i = 0
for dock_id in results:
    i += 1
    r = requests.get('https://api.tfl.gov.uk/BikePoint/'+dock_id, params=params)
    res = r.json()
    print "Station {0}: ".format(i) + str(res['commonName'])
    for e in res['additionalProperties']:
        if e['key'] == 'NbBikes':
            print "  Bikes available: " + str(e['value'])

Station 1: Claremont Square, Angel
  Bikes available: 8
Station 2: Risinghill Street, Angel
  Bikes available: 8
Station 3: Chadwell Street, Angel
  Bikes available: 1
